## Magentic-One
https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/magentic-one.html

In [ ]:
%pip install "autogen-ext[file-surfer]==0.4.0.dev13"

## Load Azure Configuration

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


# Helper Method to see the responses per agent

In [4]:
def get_agent_responses(response):
    # ANSI escape code for bold text
    bold_start = "\033[1m"
    bold_end = "\033[0m"

    # ANSI escape code for red text
    red_start = "\033[31m"
    red_end = "\033[0m"

    print(f"{bold_start}****************Agent Responses****************{bold_end}")

    for messages in response.messages:
        source = messages.source
        print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
        print(messages.content)

## Magentic-One with Assistant Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer


travel_agent = AssistantAgent(
    "TravelAgent",
    description="This agent provides a travel plan that includes travel destinations, flights, restaurants, and accommodations.",
    model_client=az_model_client,
    system_message="""
        You are a travel agent that will provide travel plans that in include travel destinations, flights, restaurants, and accommodations.
    """,
)

team = MagenticOneGroupChat([travel_agent], model_client=az_model_client)

task = """
    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.
"""
response = await Console(team.run_stream(task=task))



In [ ]:
get_agent_responses(response)

## Magentic-One with a MultimodalWebSurfer

In [ ]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat([surfer], model_client=az_model_client)
response = await Console(team.run_stream(task="What was the latest trade news on the Los Angeles Lakers basketball team?"))

In [ ]:
get_agent_responses(response)

## Magentic-One with a Code Executor Agent

In [3]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

# Create a code executor agent that uses a Docker container to execute code.
code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
await code_executor.start()
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

team = MagenticOneGroupChat([code_executor_agent], model_client=az_model_client, max_turns=50)
task = f"""
You have been provided with a dataset named bigfootsightings.csv
Use C# in a Docker container to answer the following questions:
If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar

"""
response = await Console(team.run_stream(task=task))

# Stop the code executor.
await code_executor.stop()

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:149: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:160: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)


---------- user ----------

You have been provided with a dataset named bigfootsightings.csv
Use C# in a Docker container to answer the following questions:
If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv
Use C# in a Docker container to answer the following questions:
If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar




To 

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:284: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, json_output=True)


---------- MagenticOneOrchestrator ----------
Create a Dockerfile with the necessary instructions to set up the C# environment and execute the script. Then execute shell scripts to build and run the Docker container.
---------- code_executor ----------
No code blocks found in the thread. Please provide at least one markdown-encoded code block to execute (i.e., quoting code in ```python or ```sh code blocks).
---------- MagenticOneOrchestrator ----------
Create a Dockerfile with the necessary instructions to set up the C# environment and execute the script. Then execute shell scripts to build and run the Docker container.
---------- code_executor ----------
No code blocks found in the thread. Please provide at least one markdown-encoded code block to execute (i.e., quoting code in ```python or ```sh code blocks).
---------- MagenticOneOrchestrator ----------
Please create the Dockerfile and shell scripts with the necessary instructions to set up the C# environment and execute the tasks.

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:377: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv
Use C# in a Docker container to answer the following questions:
If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code blocks), or sh shell scripts (provided to it quoted in ```sh code blocks).


Here is an initial fact sheet to consider:

Here is the updated fact sheet:

1. GIVEN OR VERIFIED FACTS
- Dataset name: bigfootsightings.csv
- Task involves handling rows that represent sigh

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:387: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------
Please run the following shell commands to create the Program.cs, Dockerfile, and build and run scripts:
```sh
# Create Program.cs
echo "using System;
using System.Globalization;
using System.IO;
using System.Linq;
using CsvHelper;
using LiveCharts;
using LiveCharts.Wpf;

class Program
{
    static void Main(string[] args)
    {
        var csvFile = \"bigfootsightings.csv\";
        var sightings = File.ReadAllLines(csvFile)
            .Skip(1)
            .Select(line => new
            {
                date = DateTime.ParseExact(
                    line.Split(',')[0],
                    \"yyyy-MM-dd\",
                    CultureInfo.InvariantCulture)
            })
            .GroupBy(x => x.date.Year)
            .Select(g => new { Year = g.Key, Count = g.Count() })
            .OrderByDescending(g => g.Count)
            .Take(10)
            .ToList();

        Console.WriteLine(\"Top 10 Years with Most Bigfoot Sightings:\");
  

Error processing publish message
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 397, in _process_publish
    await asyncio.gather(*responses)
  File "/home/codespace/.python/current/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 389, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/current/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 48, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/cu

---------- Summary ----------
Number of messages: 12
Finish reason: None
Total prompt tokens: 0
Total completion tokens: 0
Duration: 67.93 seconds


In [ ]:
get_agent_responses(response)


## Magentic-One with File Surfer

In [ ]:
from autogen_ext.agents.file_surfer._file_surfer import FileSurfer
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

file_path = "Data/nasabooks/"
# Create a file surfer agent.
file_surfer = FileSurfer(
    "FileSurfer",
    model_client=az_model_client
)
file_surfer._browser.open_path(file_path)


team = MagenticOneGroupChat(
    [file_surfer],
    model_client=az_model_client, 
    max_turns=50,)

task = """What can you see in Mauritania?"""
response = await Console(team.run_stream(task=task))


In [ ]:
get_agent_responses(response)